In [2]:
import os
import json
import joblib
import pandas as pd
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score
from lightgbm import LGBMClassifier

try:
    # Opcional en local
    from dotenv import load_dotenv
    load_dotenv(".env", override=True)
except Exception:
    pass


ARTIFACT_DIR = "model/artifacts"
MODEL_PATH = os.path.join(ARTIFACT_DIR, "rawg_lgbm_success.joblib")
METRICS_PATH = os.path.join(ARTIFACT_DIR, "metrics.json")

SUCCESS_DEFINITION = {
    "type": "quality_plus_traction",
    "rule": "is_success = 1 if (rating >= 4.0 AND ratings_count >= 100) else 0",
    "rating_threshold": 4.0,
    "ratings_count_threshold": 100
}


def get_engine():
    db_user = os.getenv("DB_USER")
    db_pass = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT", "5432")
    db_name = os.getenv("DB_NAME")

    missing = [k for k, v in {
        "DB_USER": db_user,
        "DB_PASSWORD": db_pass,
        "DB_HOST": db_host,
        "DB_NAME": db_name
    }.items() if not v]

    if missing:
        raise ValueError(f"Faltan variables de entorno: {', '.join(missing)}")

    return create_engine(
        f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
    )


def main():
    os.makedirs(ARTIFACT_DIR, exist_ok=True)

    engine = get_engine()

    # Cargamos dataset desde la VIEW (ya trae target + features preparadas)
    df = pd.read_sql("SELECT * FROM public.rawg_train_dataset;", engine)

    if "is_success" not in df.columns:
        raise RuntimeError("La vista public.rawg_train_dataset no contiene la columna is_success")

    # Tipos
    df["is_success"] = df["is_success"].astype(int)

    # Columnas que NO van en features (leakage + identificadores)
    drop_cols = ["id", "slug", "name", "released", "updated", "is_success", "rating", "ratings_count"]

    feature_cols = [c for c in df.columns if c not in drop_cols]

    # Asegurar que solo usamos numéricas (la VIEW ya intenta devolver numéricas)
    X = df[feature_cols].copy()
    for c in X.columns:
        X[c] = pd.to_numeric(X[c], errors="coerce")

    y = df["is_success"]

    # Split estratificado
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Pipeline: imputación + modelo
    pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("model", LGBMClassifier(
            n_estimators=800,
            learning_rate=0.03,
            num_leaves=31,
            subsample=0.85,
            colsample_bytree=0.85,
            objective="binary",
            random_state=42,
            n_jobs=-1
        ))
    ])

    pipe.fit(X_train, y_train)

    prob = pipe.predict_proba(X_test)[:, 1]
    pred = (prob >= 0.5).astype(int)

    report = classification_report(y_test, pred, digits=3, output_dict=True)
    auc = float(roc_auc_score(y_test, prob))

    # Guardar artefacto completo para FastAPI
    joblib.dump(
        {
            "pipeline": pipe,
            "feature_cols": feature_cols,
            "threshold": 0.5,
            "success_definition": SUCCESS_DEFINITION
        },
        MODEL_PATH
    )

    metrics = {
        "rows": int(df.shape[0]),
        "positive_rate": float(y.mean()),
        "roc_auc": auc,
        "classification_report": report,
        "success_definition": SUCCESS_DEFINITION,
        "feature_cols": feature_cols
    }

    with open(METRICS_PATH, "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)

    print("✅ Modelo guardado en:", MODEL_PATH)
    print("✅ Métricas guardadas en:", METRICS_PATH)
    print("📈 ROC-AUC:", auc)
    print("✅ Features usadas:", feature_cols)


if __name__ == "__main__":
    main()
    

[LightGBM] [Info] Number of positive: 890, number of negative: 81856
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002478 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1672
[LightGBM] [Info] Number of data points in the train set: 82746, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.010756 -> initscore=-4.521495
[LightGBM] [Info] Start training from score -4.521495


c:\Users\Nitropc\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


✅ Modelo guardado en: model/artifacts\rawg_lgbm_success.joblib
✅ Métricas guardadas en: model/artifacts\metrics.json
📈 ROC-AUC: 0.9970816080387915
✅ Features usadas: ['metacritic', 'rating_top', 'added', 'reviews_text_count', 'suggestions_count', 'reddit_count', 'twitch_count', 'youtube_count', 'release_year', 'release_month', 'days_since_release']
